This script outputs a text file suitable for import to [https://dbdiagram.io](https:\dbdiagram.io\) , using the DBML language. [https://dbml.dbdiagram.io/docs](https:\dbml.dbdiagram.io\docs).  
It outputs datatypes expected to be found in an SQL server instance, and is expecting SQL server type information from the database.

\- get list of attributes in target structures from attributes table

\- get list of FKs from link table (not implemented yet)

\- spin dfs to json dicts

\- throw at jinja

In [21]:
import json
import os
import sqlite3

from pathlib import Path
from jinja2 import Template, Environment
from datetime import datetime

import pandas as pd
import numpy as np

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

template_path = "../export_templates/DBML/"
template_filename = "Mysql_physical_er.tem"
template = Path(os.path.join(template_path, template_filename)).read_text()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/DBML/"
output_filename = "Mysql_physical_er_" + time_string + ".txt"

In [2]:
# set variables for the extract
# A target schema can be set to '' to get all schemas in the DB

target_server_name = 'DRS_Core'
target_db_name = 'def'
target_schema_name = 'optitimeweb_sync' # a value or '' (blank)


In [11]:
# get list of attributes in target structures from attributes table

if (target_schema_name != ''):
  target_schema_phrase = """AND "SCHEMA_NAME" = ?"""
else:
  target_schema_phrase = ''

sql_query_core = """
SELECT
  '"' || SCHEMA_NAME || '"."' || TABLE_NAME || '"' AS "TableKeyPhrase"
	, SCHEMA_NAME
	, TABLE_NAME
	, COLUMN_NAME
	, IS_NULLABLE
	, DATA_TYPE
	, CHARACTER_MAXIMUM_LENGTH
	, NUMERIC_PRECISION
	, NUMERIC_SCALE
FROM
	out_MysqlPhysicalAttributeOutputForAnalysis
WHERE
	1=1
	AND "SERVER_NAME" = ?
	AND "DATABASE_NAME" = ?
""";
sql_query_order = """
ORDER BY
	SCHEMA_NAME
	, TABLE_NAME
	, ORDINAL_POSITION
""";

sql_query = sql_query_core + target_schema_phrase + sql_query_order

if (target_schema_name != ''):
  sql_parameters = (target_server_name, target_db_name, target_schema_name)
else:
  sql_parameters = (target_server_name, target_db_name)

dtypeDict = {
    'CHARACTER_MAXIMUM_LENGTH': 'Int64'
  , 'NUMERIC_PRECISION': 'Int64'
  , 'NUMERIC_SCALE': 'Int64'
}
df = pd.read_sql_query(sql_query, conn, params = sql_parameters, dtype = dtypeDict)
df = df.replace({np.nan: None})

#df

In [15]:
# add tables/ columns to output dict

grouped = df.groupby(['TableKeyPhrase', 'COLUMN_NAME']).apply(lambda x: x.drop(columns=['TableKeyPhrase', 'COLUMN_NAME']).to_dict(orient='records')).to_dict()

template_data = {}
template_data['tables'] = {}

for (TableKeyPhrase, COLUMN_NAME), attributes in grouped.items():
  if TableKeyPhrase not in template_data['tables']:
    template_data['tables'][TableKeyPhrase] = {}
  template_data['tables'][TableKeyPhrase][COLUMN_NAME] = attributes

#template_data

In [17]:
# get list of FKs from link table

if (target_schema_name != ''):
  target_schema_phrase = """AND SCHEMA_NAME_CHILD = ? AND SCHEMA_NAME_PARENT = ?"""
else:
  target_schema_phrase = ''

sql_query_core = """
SELECT
  SCHEMA_NAME_CHILD
  , TABLE_NAME_CHILD
  , COLUMN_NAME_CHILD
  , SCHEMA_NAME_PARENT
  , TABLE_NAME_PARENT
  , COLUMN_NAME_PARENT
FROM
  out_PhysicalForeignKeys
WHERE
  1=1
  AND SERVER_NAME_CHILD = ?
  AND DATABASE_NAME_CHILD = ?
  AND SERVER_NAME_PARENT = ?
  AND DATABASE_NAME_PARENT = ?

""";

sql_query = sql_query_core + target_schema_phrase

if (target_schema_name != ''):
  sql_parameters = (target_server_name, target_db_name, target_server_name, target_db_name, target_schema_name, target_schema_name)
else:
  sql_parameters = (target_server_name, target_db_name, target_server_name, target_db_name)

df = pd.read_sql_query(sql_query, conn, params = sql_parameters)
df = df.replace({np.nan: None})

#df

In [23]:
# add FKs to output dict

template_data['FKrecords'] = df.to_dict(orient='records')

#print(template_data['FKrecords'])

In [22]:
# throw at jinja

j2_template = Template(template, autoescape=True, trim_blocks=True, lstrip_blocks=True)

rendered_template_string = j2_template.render(template_data=template_data)
with open(os.path.join(output_path, output_filename), "w") as text_file:
  text_file.write(rendered_template_string)